# GIF CREATION FOR Ground Navigation for Aerial Vehicles

With a satellite reference image, we intend to utilize a SfM solution to match ground segments of images to the reference map.
- Inputs: Reference image (satellite), SfM solution (images, points, cameras), selected images (5)
- Output: Correction solutions for translation, rotation, scaling

In [1]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from scipy.spatial.transform import Rotation as R
from scipy.spatial import cKDTree
import imageio
import time
# %matplotlib qt
import matplotlib.pyplot as plt
from matplotlib.path import Path

from groundNAV_agent import *
from colmapParsingUtils import *

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# # Load in necessary parameters for gNAV agent 
# # Define Class Parameters 

# images_colm  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
# cameras_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
# pts3d_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"

# # Images selected for local corrections
# image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG" #ID:4
# image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9520.JPEG" #ID:78
# image_3 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9531.JPEG" #ID:89
# image_4 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9542.JPEG" #ID:100
# image_5 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9576.JPEG" #ID:134
# # Load in satellite reference image
# sat_ref = "TTurf/TurfSat.jpg"
# # sat_ref = cv2.imread('TTurf/TurfSat.jpg')

# # Organize for agent params
# images = [image_1, image_2, image_3, image_4, image_5]

In [3]:
# CLOSER IMAGES 
# Load in necessary parameters for gNAV agent 
# Define Class Parameters 

images_colm  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
cameras_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
pts3d_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"

# Images selected for local corrections
# image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9449.JPEG"
image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9475.JPEG"
# image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9459.JPEG"
image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9464.JPEG"
image_3 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9467.JPEG"
image_4 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9473.JPEG"
image_5 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9476.JPEG"
# Load in satellite reference image
sat_ref = "TTurf/TurfSat.jpg"
# sat_ref = cv2.imread('TTurf/TurfSat.jpg')

# Organize for agent params
images = [image_1, image_2, image_3, image_4, image_5]

In [4]:
# Create class
gnav = gNAV_agent(images_colm, cameras_colm, pts3d_colm, images, sat_ref)

# Grab raw points and RGB data for scene and reference cloud
scene_pts, rgb_data = gnav.grab_pts(gnav.pts3d_c)
ref_pts, ref_rgb = gnav.ref_pts, gnav.ref_rgb

### Changing REF FRAME TO MIDDLE

In [5]:
# Use ground plane pts to set reference frame 
# Need gravity and height
pts_gnd_idx = np.array([25440, 25450, 25441, 25449, 25442, 25445, 103922, 103921, 103919, 103920])
# tform_ref_frame = gnav.set_ref_frame(pts_gnd_idx) # THIS IS WHAT I AM CHANGING 
tform_ref_frame = gnav.set_ref_frame_mid(pts_gnd_idx) # NEW VERSION
tform_ref_frame_pts = gnav.inv_homog_transform(tform_ref_frame)
print("\nReference frame transformation\n", tform_ref_frame_pts)


Reference frame transformation
 [[-1.55069060e-03  9.81197008e-01  1.93002661e-01 -1.21025836e-01]
 [-1.42845166e-01 -1.91240997e-01  9.71093270e-01  1.86102525e+00]
 [ 9.89743833e-01 -2.60636319e-02  1.40455805e-01  7.28134156e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# Transform all points to the new coordinate system 
# Not necessary since we aren't using the cloud, but a good visual check for coord frame
tform_ref_frame_inv = gnav.inv_homog_transform(tform_ref_frame)
origin_ref, scene_pts_ref, scene_vec_ref = gnav.unit_vec_tform(scene_pts, gnav.origin_w, tform_ref_frame_inv)
# print(origin_ref)
# Transform scene cloud to 2D (also as a visual check)
# Note: 2d projection will look off with z=-1; see TTurf_v2 for cropping method
scene_ref_2d = gnav.proj_2d_scene(scene_pts_ref)
# print(scene_ref_2d)

In [ ]:
# Image 1 - OS in Jumbos OR top half of S

# # OS in Jumbos 
# imnum = 0
# # x,y = 150,1725
# # side_x = 2250 # WIDTH
# # side_y = 750 # HEIGHT
# x,y = 150, 2000
# side_x = 1500
# side_y = 250 # HEIGHT

# # Top half of S
# imnum = 0
# x,y = 300,2100
# side_x = 1300 # WIDTH
# side_y = 750 # HEIGHT
# x,y = 300,2200
# side_x = 1300 # WIDTH
# side_y = 600 # HEIGHT

# # Bottom of M 
# imnum = 0
# x,y = 1100,2000
# side_x = 1300 # WIDTH
# side_y = 600 # HEIGHT

# # 10 yd hash 
imnum = 0
x,y = 1500,1000
side_x = 900 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_loc)
# print(pts_rgb)

In [ ]:
# Image 2 - 40yd (far), OR O in jumbos

# # 40yd (far)
# imnum = 1
# x,y = 1150,1150
# side_x = 1100 # WIDTH
# side_y = 900 # HEIGHT

# # O in jumbos 
# imnum = 1
# x,y = 300,1700
# side_x = 1000 # WIDTH
# side_y = 900 # HEIGHT

# Bottom of S
imnum = 1
x,y = 1400,2300
side_x = 800 # WIDTH
side_y = 1200 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [ ]:
# Image 3 - Jumbo logo OR B in jumbos

# # Jumbo logo
# imnum = 2
# x,y = 400,1500
# side_x = 1500 # WIDTH
# side_y = 750 # HEIGHT

# B in jumbos 
imnum = 2
x,y = 800,2100
side_x = 1400 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [ ]:
# Image 4 - 30yd (near) OR edge to endzone 

# 30yd (near)
# imnum = 3
# x,y = 1350,1450
# side_x = 1200 # WIDTH
# side_y = 500 # HEIGHT

# Edge to endzone
imnum = 3
x,y = 1150,900
side_x = 1200 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [ ]:
# Image 5 - 10yd marker
imnum = 4
x,y = 1200,1300
side_x = 1000 # WIDTH
side_y = 700 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [ ]:
# Generate projection of image sections 
for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    # Unit vectors in camera coords 
    pts_vec_c, pts_rgb_gnd = gnav.unit_vec_c(i)
    gnav.im_mosaic[i] = {'rgbc': pts_rgb_gnd}

    # Get transformation matrix that move from camera coords to world coords
    id = gnav.im_ids[i]
    homog_w2c, homog_c2w = gnav.get_pose_id(id,i)
    # print('Homogeneous transformation from world to camera \n', homog_c2w)
    # print('\n Homogeneous transformation from camera to world \n', homog_w2c)

    # Transform to world coords
    origin_c, pts_loc_w, pts_vec_w = gnav.unit_vec_tform(pts_vec_c, gnav.origin_w, homog_c2w)
    # print('\n New camera frame origin = ', origin_c)
    
    # Get new points 
    ranges, new_pts_w = gnav.pt_range(pts_vec_w, homog_c2w, origin_c, i)
    # print('\nNew Points \n', new_pts_w)

    # Transfer points to reference frame
    __, new_pts_r, pts_vec_r = gnav.unit_vec_tform(new_pts_w, gnav.origin_w, tform_ref_frame_pts)

    # Convert points to grayscale 
    gray_c = gnav.conv_to_gray(gnav.im_mosaic[i]['rgbc'],i)
    # print(gray_c)

    # Put new points and grayscale colors in image mosaic
    gnav.im_mosaic[i]['pts'] = new_pts_r
    gnav.im_mosaic[i]['color_g'] = gray_c
    
    print("\nDone image ", i)

In [ ]:
# OLD GUESSES 


# scale = 87 # Avg guess OLD IMAGES
# scale = 80 # Decent average guess - Old frame, new ims

# # Avg guess for OLD IMAGES  
# x = 294
# y = 172
# yaw = np.deg2rad(1)

# # Going for average guess - new images!, old frame
# x = 331
# y = 264
# yaw = np.deg2rad(-1)
# # Corrected guess (just for 10yd line)! 
# x = 333
# y = 260
# yaw = np.deg2rad(-1)

In [ ]:
# Implementing an initial guess for the local image 

# SCALE for initial guess 
# scale = gnav.focal/39
scale = 80

# Avg guess for OLD IMAGES  
# x = -55
# y = 20
# yaw = np.deg2rad(140)
x = -50
y = 15
yaw = np.deg2rad(135)


tform_guess = gnav.tform_create(x,y,0,0,0,yaw)
gnav.best_guess_tform = tform_guess
gnav.best_guess_scale = scale
# print(tform_guess)

for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    loc_im_pts = gnav.im_mosaic[i]['pts'].copy()
    # print(loc_im_pts)
    loc_im_pts[:, :2] *= scale
    # Get new points 
    __, loc_im_pts_guess, loc_im_vec_guess = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_guess)
    gnav.im_pts_best_guess[i] = {'pts': loc_im_pts_guess}
    # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 

    print("\nDone image ", i)

In [ ]:
# PLOTTING THE NEW SCENE MOSAIC

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Mosaic scene with satellite reference")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40)

# Create point cloud for image points
im0_cloud = o3d.geometry.PointCloud()
im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[0]['pts'])
im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])

# Create point cloud for image points
im1_cloud = o3d.geometry.PointCloud()
im1_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[1]['pts'])
im1_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['color_g'])

# Create point cloud for image points
im2_cloud = o3d.geometry.PointCloud()
im2_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[2]['pts'])
im2_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['color_g'])

# Create point cloud for image points
im3_cloud = o3d.geometry.PointCloud()
im3_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[3]['pts'])
im3_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['color_g'])

# Create point cloud for image points
im4_cloud = o3d.geometry.PointCloud()
im4_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[4]['pts'])
im4_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['color_g'])

# Create point cloud for reference cloud (satellite)
ref_cloud = o3d.geometry.PointCloud()
ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)

# Add necessary geometries to visualization 
vis.add_geometry(axis_origin)
vis.add_geometry(ref_cloud)
vis.add_geometry(im0_cloud)
vis.add_geometry(im1_cloud)
vis.add_geometry(im2_cloud)
vis.add_geometry(im3_cloud)
vis.add_geometry(im4_cloud)


# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Set up initial viewpoint
view_control = vis.get_view_control()
# Direction which the camera is looking
view_control.set_front([0, 0.01, -1])  # Set the camera facing direction
# Point which the camera revolves about 
view_control.set_lookat([0, 0, 0])   # Set the focus point
# Defines which way is up in the camera perspective 
view_control.set_up([0, -1, 0])       # Set the up direction
view_control.set_zoom(.45)           # Adjust zoom if necessary




# Run and destroy visualization 
# vis.run()
vis.poll_events()
vis.update_renderer()
vis.capture_screen_image("Figs_gifs/Mapgifs/May6Test_10x10Rot/screenshot_0.png")
time.sleep(1.5)  # Keep window open for 1.5 seconds
vis.destroy_window()
# vis.destroy_window()


### SSD Process (using original version)

In [ ]:
# n = 5
# for imnum in range(len(images)):
# # Just for the first image for now 
# # for imnum in range(1):
# # for imnum in range(4, 5, 1):
#     ssds = gnav.ssd_nxn(n, imnum)
#     gnav.ssds_curr[imnum] = ssds
# print("Done, most recent SSDs are\n", ssds)

In [ ]:
# # NEW new LOOKUP STRATEGY - SHOULD BE FASTER - NOT WORKING (yet)
# n = 5
# for imnum in range(len(images)):
# # Just for the first image for now 
# # for imnum in range(1):
#     ssds1 = gnav.ssd_nxn_NEWnew(n, imnum)
#     gnav.ssds1_curr[imnum] = ssds1
# print("Done, SSDs are\n", ssds1)

# Least Square Process

### Create a vector for each SSD

### Full Lsquares 

In [ ]:
# Initial guess parameters
# INITIAL GUESS Parameters
# s = 1
# theta = np.deg2rad(0)
# tp = 0
# tq = 0
s = scale
theta = yaw
tp = x
tq = y
params = np.array([s, theta, tp, tq]).reshape(4,1)
init_guess = np.array([s, theta, tp, tq]).reshape(4,1)
print("\nInitial Guess parameters\n", params)

In [ ]:
iterations = 5
n = 10

viss = {}

# for iter_idx in range(5,10):
for iter_idx in range(iterations):
    # Loop through full set of images
    for imnum in range(len(images)):
        ssds = gnav.ssd_nxn(n, imnum)
        gnav.ssds_curr[imnum] = ssds
    # print("Most recent SSDs\n", ssds)
    

    
    
    # Create a vector from the original position to the minimum SSD location 
    cor_vecs = np.zeros((len(images), 2))
    base_vec = np.zeros((len(images), 2))
    for im_cv in range(len(images)):
    # for imnum in range(1):
        # Grab SSDs - get id of minimum
        ssds = gnav.ssds_curr[im_cv]
        # print(f"\nSSDS for image {i}\n", ssds)
        idrow, idcol = np.unravel_index(np.argmin(ssds), ssds.shape)
        # print("\nidrow, idcol\n", idrow, idcol)
        # Define best shift vector 
        shiftx_min = idrow-n
        shifty_min = idcol-n
        print("\nBEST SHIFT VECTOR = ", shiftx_min, shifty_min)
        print(f"\nBEST SSD for image {im_cv} = ", ssds[idrow, idcol])
        cor_vecs[im_cv] = shiftx_min, shifty_min
        # Get mean of satellite points for base of x and y
        sat_pts, __ = gnav.get_inside_sat_pts(im_cv, 0,0)#shiftx_min, shifty_min)
        # print("\nInside satellite points\n", sat_pts)
        basex, basey = np.mean(sat_pts[:,0]), np.mean(sat_pts[:,1])
        # print("\nBase of x and y = ", basex, basey)
        base_vec[im_cv] = basex, basey
    
    print("\nCorrection vectors:\n", cor_vecs)
    print("\nBase of correction vectors: \n", base_vec)
    
    points_b = np.hstack((base_vec, np.zeros((len(images), 1))))
    points_e = points_b + np.hstack((cor_vecs, np.zeros((len(images),1))))
    points = np.vstack((points_b, points_e))
    # print("\nBeginning of points: \n", points_b)
    # print("\nEnd of points: \n",points_e)
    # print("\nAll points: \n",points)
    lines = []
    for lin in range(len(images)):
        lines.append([lin,lin+len(images)])
        # print(lin, lin+len(images))
    
    # print("\nLines for OPEN3d:\n",lines)
    
    # Delta y term
    # print(cor_vecs.reshape(-1,1))
    yi = cor_vecs.reshape(-1,1)
    print("\nYi\n", yi)
    
    
    
    
    
    # Form jacobian for each image
    J = np.zeros((2*len(images),4))
    theta = params[1][0]
    s = params[0][0]
    # print(J)
    for i_m in range(len(images)):
        xpi = np.mean(gnav.im_pts_best_guess[i_m]['pts'][:,0])
        xqi = np.mean(gnav.im_pts_best_guess[i_m]['pts'][:,1])
        # print(xpi, xqi)
        # Jacobian values (2x4)
        j11 = np.cos(theta)*xpi - np.sin(theta)*xqi
        j21 = np.sin(theta)*xpi + np.cos(theta)*xqi
        j12 = -params[0][0]*(np.sin(theta)*xpi + np.cos(theta)*xqi)
        j22 = params[0][0]*(np.cos(theta)*xpi - np.sin(theta)*xqi)
        j13, j23, j14, j24 = 1, 0, 0, 1
        # print(j13)
    
        # Construct Jacobian 
        J_upper = np.hstack((j11, j12, j13, j14))  # First row block (y_p terms)
        # print("\nJ Upper\n", J_upper)
        J_lower = np.hstack((j21, j22, j23, j24))  # Second row block (y_q terms)
        # print("\nJ Lower\n", J_lower)
        j = np.vstack((J_upper, J_lower))  # Shape (2N, 4)
        print("\nJacobian\n", j)
        J[2*(i_m):2*(i_m)+2, :] = j
    
    print("\nJACOBIAN\n", J)
    # print("\nJacobian shape: ", J.shape)
    
    # Least squares process
    JTJi = np.linalg.inv(J.T@J)
    Dalpha = JTJi@J.T@yi
    print("\nDelta Alpha:\n", Dalpha)
    
    
    
    params += Dalpha
    print("\nUpdated Params: scale, theta, tq, tp\n", params)
    
    
    
    # Apply change
    tform_mat = gnav.tform_create(params[2][0], params[3][0], 0, 0, 0, params[1][0]) # x,y,z,roll,pitch,yaw
    print("\nTransformation matrix\n", tform_mat)
    
    
    for i_c in range(len(images)):
    # Just for the first image for now
    # for i in range(1):
        loc_im_pts = gnav.im_mosaic[i_c]['pts'].copy()
        # print(loc_im_pts)
        loc_im_pts[:, :2] *= params[0][0].astype(np.float64)
        # Get new points 
        __, loc_im_pts_NEW, __ = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_mat)
        gnav.im_pts_best_guess[i_c]['pts'] = loc_im_pts_NEW
        # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 
    
        # print("\nDone image ", i)








    # PLOT NEWEST GUESS 
    globals()[f"vis_{iter_idx}"] = o3d.visualization.Visualizer()
    globals()[f"vis_{iter_idx}"].create_window(window_name="Mosaic scene with satellite reference")
    
    # Create axes @ origin
    axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40)
    
    # Create point cloud for image points
    im0_cloud = o3d.geometry.PointCloud()
    im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[0]['pts'])
    im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])
    
    # Create point cloud for image points
    im1_cloud = o3d.geometry.PointCloud()
    im1_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[1]['pts'])
    im1_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['color_g'])
    
    # Create point cloud for image points
    im2_cloud = o3d.geometry.PointCloud()
    im2_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[2]['pts'])
    im2_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['color_g'])
    
    # Create point cloud for image points
    im3_cloud = o3d.geometry.PointCloud()
    im3_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[3]['pts'])
    im3_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['color_g'])
    
    # Create point cloud for image points
    im4_cloud = o3d.geometry.PointCloud()
    im4_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[4]['pts'])
    im4_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['color_g'])
    
    # Create point cloud for reference cloud (satellite)
    ref_cloud = o3d.geometry.PointCloud()
    ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
    ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)
    
    # Add necessary geometries to visualization 
    globals()[f"vis_{iter_idx}"].add_geometry(axis_origin)
    globals()[f"vis_{iter_idx}"].add_geometry(im0_cloud)
    globals()[f"vis_{iter_idx}"].add_geometry(im1_cloud)
    globals()[f"vis_{iter_idx}"].add_geometry(im2_cloud)
    globals()[f"vis_{iter_idx}"].add_geometry(im3_cloud)
    globals()[f"vis_{iter_idx}"].add_geometry(im4_cloud)
    globals()[f"vis_{iter_idx}"].add_geometry(ref_cloud)
    
    # # Size options (jupyter gives issues when running this multiple times, but it looks better)
    # render_option = vis.get_render_option()
    # render_option.point_size = 2

    # Set up initial viewpoint
    view_control = globals()[f"vis_{iter_idx}"].get_view_control()
    # Direction which the camera is looking
    view_control.set_front([0, 0.01, -1])  # Set the camera facing direction
    # Point which the camera revolves about 
    view_control.set_lookat([0, 0, 0])   # Set the focus point
    # Defines which way is up in the camera perspective 
    view_control.set_up([0, -1, 0])       # Set the up direction
    view_control.set_zoom(.45)           # Adjust zoom if necessary

    # viss[iter_idx] = globals()[f"vis_{iter_idx}"]
    # Run and destroy visualization 
    # vis.run()
    globals()[f"vis_{iter_idx}"].poll_events()
    globals()[f"vis_{iter_idx}"].update_renderer()
    globals()[f"vis_{iter_idx}"].capture_screen_image(f"Figs_gifs/Mapgifs/May6Test_10x10Rot/screenshot_{iter_idx+1}.png")
    time.sleep(1.5)  # Keep window open for 1.5 seconds
    globals()[f"vis_{iter_idx}"].destroy_window()    
    


### Plot newest guess

In [ ]:
# # PLOTTING THE NEW SCENE MOSAIC

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene with satellite reference")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40)

# # Create point cloud for image points
# im0_cloud = o3d.geometry.PointCloud()
# im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[0]['pts'])
# im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])

# # Create point cloud for image points
# im1_cloud = o3d.geometry.PointCloud()
# im1_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[1]['pts'])
# im1_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['color_g'])

# # Create point cloud for image points
# im2_cloud = o3d.geometry.PointCloud()
# im2_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[2]['pts'])
# im2_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['color_g'])

# # Create point cloud for image points
# im3_cloud = o3d.geometry.PointCloud()
# im3_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[3]['pts'])
# im3_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['color_g'])

# # Create point cloud for image points
# im4_cloud = o3d.geometry.PointCloud()
# im4_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[4]['pts'])
# im4_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['color_g'])

# # Create point cloud for reference cloud (satellite)
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(im0_cloud)
# vis.add_geometry(im1_cloud)
# vis.add_geometry(im2_cloud)
# vis.add_geometry(im3_cloud)
# vis.add_geometry(im4_cloud)
# vis.add_geometry(ref_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2



# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

# PLOTTING TOOLS 

In [ ]:
# # Plot and visualize SSD values (was originally used in LSquares Loop 

# # Create a 5x5 grid of x and y coordinates
# x = np.linspace(-n, n, 2*n+1)
# y = np.linspace(-n, n, 2*n+1)
# Y, X = np.meshgrid(x, y)
# # print(ssds)

# # print(x)
# # print(X)
# # print(Y)
# i = 1
# # Best shift
# idrow, idcol = np.unravel_index(np.argmin(SSDS_CURR[i]), ssds.shape)
# print(idrow, idcol)
# shiftx_min = idrow-n
# shifty_min = idcol-n
# # print("BEST SHIFT = ", shiftx_min, shifty_min)
# # print("BEST SSD = ", SSDS_CURR[i][idrow, idcol])

# # PLOTTING A SINGLE VECTOR FIELD FOR SSD
# # Create the figure and 3D axis
# %matplotlib qt
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Plot the 3D vectors
# ax.quiver(X, Y, np.zeros_like(SSDS_CURR[i]), np.zeros_like(SSDS_CURR[i]), np.zeros_like(SSDS_CURR[i]), (SSDS_CURR[i]/1000)**2, arrow_length_ratio=0.1)

# # Set axis limits
# ax.set_xlim([-11, 11])  # X axis range
# ax.set_ylim([-11, 11])  # Y axis range
# ax.set_zlim([0, (np.max(SSDS_CURR[i]) / 1000)**2])  # Z axis range, adjust based on your data

# # Labels and title
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('Pixel Correction Vector Field')